In [6]:
import requests
import json
import pandas as pd
# import import_ipynb
import googleapi as google
from IPython.display import display, HTML, Image
from ipywidgets import interact_manual
from datetime import datetime
from dotenv import load_dotenv
import os

def config():
    load_dotenv()


weather_key = os.getenv("Weather_key")
google_api_key = os.getenv("google_api_key")



def get_geocode(location):
    return google.googleGeocode(google_api_key, location)



In [7]:
def get_weather_data(weather_key, lat, lon):
    url = 'https://api.openweathermap.org/data/2.5/onecall?'
    query_string = {
        "lat": lat,
        "lon": lon,
        "exclude": "minutely,hourly",
        'units': "imperial",
        "appid": weather_key

    }
    response = requests.get(url, params=query_string)
    response.raise_for_status()
#     print(response.url)
    response = response.json()
    return response


def image_link(link):
    return '<img src="'+ link + '" width="60" >'


In [8]:

@interact_manual(Location = "")
def main(Location):
    #get geocode of location
    geo_data = get_geocode(Location)
    lat = geo_data['lat']
    lon = geo_data['lng']

    # get the weather of the loction
    weather = get_weather_data(weather_key, lat, lon)
    # Current temperature and description of the location
    temp = round(weather["current"]['temp'])
    description = weather["current"]['weather'][0]['description']
    #weather icon
    icon = weather["current"]['weather'][0]['icon']
    iconurl = f"http://openweathermap.org/img/wn/{icon}@2x.png"

    #display the current temp, description and icon
    display(HTML(f"<h2>Current Weather in {Location}<h2>"))
    display(HTML(f"<p>{description} with a Temperature of {temp} Deg F.<p>"))
    display(Image(url=iconurl))

    #Data Frame
    display(HTML(f"<h2>7 Day Forecast for {Location}<h2>"))
    #All the weather data in a data frame one
    df = pd.DataFrame(weather['daily'])

    #Turn the dict in temp into a new columns data frame 2
    df2 = pd.json_normalize(df["temp"])

    #use lambda to convert unix timestamp into datetime format (%yyyy,%dd,%mm)(1 = x-axis)
    days = df.apply(lambda x: datetime.fromtimestamp(x['dt']), axis=1  )
    # create a data frame from the days only
    days_df = pd.DataFrame(days)
    #use lambda to turn the panda Series object into an int
    df2 = df2.apply(lambda x:x.astype(int), axis=1)
    #both frames that need to be concatnated
    frames = [days_df, df2]
    result = pd.concat(frames, axis=1,)
    #Change the column names
    week_weather = result.rename(columns ={0: "Day", 'min': 'Low', 'max': "High"})
    # convert datetime into pandas day name
    iconurls = []
    current_icon = weather["current"]
    for week in weather['daily']:
        for day in week['weather']:
            icon = day['icon']
            iconurl = f"http://openweathermap.org/img/wn/{icon}@2x.png"
            iconurls.append(iconurl)


    week_weather['Icon'] = iconurls
    week_weather['Day'] = week_weather['Day'].dt.day_name()
    week_weather.apply(lambda x: image_link(x['Icon']), axis=1)

    # #grap only necessary information to display
    week_weather = week_weather[['Icon', 'Day', "Low", 'High']]


    icon_df = HTML(week_weather.to_html(escape=False,formatters=dict(Icon=image_link)))
    display(icon_df)


interactive(children=(Text(value='', description='Location'), Button(description='Run Interact', style=ButtonS…